<a href="https://colab.research.google.com/github/shintaroudlotulhanafia/DRLforMultipleStocksTradingUsingFinRL/blob/main/FractionsTest3/3_2_fraksi1_TugasAkhirS1_Shinta_DRLforMultipleStocksTradingUsingFinRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook ini dijalankan menggunakan email shintarh18@gmail.com

# Deep Reinforcement Learning untuk Jual-Beli Saham (Dari Awal): Jual-beli Banyak Saham 

* **Pytorch Version** 



# Daftar Isi:

* [1. Pendeskrisian Tugas](#0)
* [2. Instalasi Paket Python](#1)
    * [2.1. Instalasi Paket](#1.1)    
    * [2.2. Daftar Paket Python](#1.2)
    * [2.3. Impot Paket](#1.3)
    * [2.4. Membuat Folder](#1.4)
* [3. Mengunduh Data](#2)
* [4. Memproses Data](#3)        
    * [4.1. IndiKator Teknikal](#3.1)
    * [4.2. Melakukan *Feature Engineering*](#3.2)
* [5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style](#4)  
    * [5.1. Pemisahan Data](#4.1)  
    * [5.2. Lingkungan untuk Pelatihan *(Training)*](#4.2)    
* [6. Melatih Agen DRL](#5)
    * [6.1. Agen 1: A2C](#5.1)
    * [6.2. Agen 2: PPO](#5.2)
    * [6.3. Agen 3: TD3](#5.3)
    * [6.4. Agen 4: SAC](#5.4)
    * [6.5. Agen 5: DDPG](#5.5)
* [7. Melakukan Jual-Beli](#6)
    * [7.1. Performa di Dalam Sampel](#6.1)
    * [7.2. Performa di Luar Sampel](#6.2)
    * [7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi](#6.3)
      * [7.3.1. Agen 1: A2C](#6.3.1)
      * [7.3.2. Agen 2: PPO](#6.3.2)
      * [7.3.3. Agen 3: TD3](#6.3.3)
      * [7.3.4. Agen 4: SAC](#6.3.4)
      * [7.3.5. Agen 5: DDPG](#6.3.5)
* [8. Performa Backtesting](#7)  
    * [8.1. Status BackTesting](#7.1)
    * [8.2. Gambaran BackTesting](#7.2)

<a id='0'></a>
# Bagian 1. Pendeskrisian Tugas

Agen DRL dilatih untuk melakukan jual-beli saham. Tugas tersebut dimodelkan sebagai Markov Decision Process (MDP), dengan fungsi dan tujuannya adalah memaksimalkan pengembalian pengembalian kumulatif *(cummulative return)* yang diharapkan.

Definisi state-action-reward pada algoritam DRL kasus kali ini adalah sebagai berikut:

* **State s**: Kondisi atau *state* mewakili persepsi agen tentang lingkungan pasar. Sama seperti *trader* manusia yang menganalisis berbagai informasi, agen juga secara pasif mengamati banyak fitur dan belajar dengan cara berinteraksi dengan lingkungan pasar (biasanya dengan menjalankan ulang data historis).

* **Tindakan atau *action* a**: Ruang aksi mencakup aksi atau tindakan yang dapat dilakukan agen di setiap status atau kondisi. Misalnya, a {−1, 0, 1}, -1 berarti menjual, 0 berarti menahan, dan 1 berarti membeli. Ketika suatu aksi mengoperasikan beberapa saham, maka, a {−k, ..., 1, 0, 1, ..., k}. Misalnya, "Beli
10 saham TLKM" atau "Jual 10 saham TLKM" maka masing-masing nilai a adalah 10 atau -10.

* **Fungsi *reward* atau imbalan r(s, a, s′)**: *Reward* adalah insentif bagi agen untuk mempelajari kebijakan yang lebih baik. Misalnya *reward* dapat berupa perubahan nilai portofolio saat mengambil a pada keadaan s dan tiba pada keadaan baru s', yaitu, r(s, a, s′) = v′-v, v′ mewakili nilai portofolio pada keadaan s′ dan v mewakili nilai portofolio pada keadaan s.

* **Environment atau lingkungan jual-beli**: saham penyusun indeks JII dengan tanggal sesuai periode pengujian yang diatur.


Data untuk studi kasus ini diperoleh dari Yahoo Finance API. Data berisi harga *Open-High-Low-Close* dan *Volume*.

<a id='1'></a>
# Bagian 2. Instalasi Paket Python

<a id='1.1'></a>
## 2.1. Instalasi Paket


In [1]:
# install finrl library
!pip install git+https://github.com/shintaroudlotulhanafia/FinRL.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/shintaroudlotulhanafia/FinRL.git to /tmp/pip-req-build-nr9lm618
  Running command git clone -q https://github.com/shintaroudlotulhanafia/FinRL.git /tmp/pip-req-build-nr9lm618
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-uc6floc4/pyfolio_147a61ca75fd48cb8778a0a72a2b387a
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-uc6floc4/pyfolio_147a61ca75fd48cb8778a0a72a2b387a
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-uc6floc4/elegantrl_1081ff419f014c17aeffcaac77dff314
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-uc6floc4/elegantrl_1081ff419f014c17aeffcaac77dff314
     |████████████████████████████████| 2.3 MB 56.5 MB/s 
     |████████████████████████████████| 234 kB 77.7 MB/s 
     |█████████████████████

<a id='1.2'></a>
## 2.2. Daftar Paket Python
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Impot Paket

* Modul Python **pandas** digunakan untuk menganalisis dan memanipulasi data.
* Modul Python **numpy** digunakan untuk memproses larik atau array.
* Modul Python **matplotlib** digunakan membuat visualisasi data dalam dua dimensi.
* Modul Python **matplotlib.pyplot** adalah kumpulan fungsi yang membuat matplotlib berfungsi seperti MATLAB.
* Modul Python **Datetime** menyediakan sejumlah fungsi untuk menangani tanggal, waktu, dan interval waktu. Date dan datetime adalah objek dalam Python, bukan string atau timestamps.
* **YahooDownloader** menyediakan metode untuk mengambil data saham harian dari API Keuangan Yahoo!
* **FeatureEngineer** menyediakan metode untuk preprocessing data harga saham
* **data_split** membagi dataset menjadi data pelatihan dan data pengujian berdasarkan tanggal
* **StockTradingEnv** Lingkungan perdagangan saham untuk OpenAI gym
* **DRLAgent** menyediakan implementasi untuk algoritma DRL
* **DataProcessor** memproses data menggunakan prosesor data terpadu
* **backtest_stats** menghitung statistik *backtesting*
* **backtest_plot** membuat dan menampilkan plot ringkasan laporan *backtesting*.
* **get_daily_return** 
* **get_baseline** mengunduh data berdasarkan suatu indeks pada periode waktu tertentu.
* Metode **sys.path.append()** digunakan untuk menambahkan jalur sementara. Dengan demikian, jalur tersebut akan valid untuk sebuah sesi, misalnya.
* Python **itertool** adalah modul yang menyediakan berbagai fungsi yang bekerja pada iterator untuk menghasilkan iterator yang kompleks. Modul ini berfungsi sebagai alat yang cepat dan hemat memori.

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


<a id='1.4'></a>
## 2.4. Membuat Folder

In [3]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

<a id='2'></a>
# Part 3. Mengunduh Data

Yahoo Finance menyediakan data saham, berita keuangan, laporan keuangan, dll, secara gratis.
* FinRL *Library* menggunakan kelas **YahooDownloader** di FinRL-Meta untuk mengambil data melalui Yahoo Finance API
* Batas Pemanggilan: Menggunakan API Publik (tanpa autentikasi), Pengguna dibatasi hingga 2.000 permintaan per jam per IP (atau hingga total 48.000 permintaan per hari).

In [4]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2017-12-31'
TRADE_START_DATE = '2018-01-01'
TRADE_END_DATE = '2019-12-20'

In [5]:
#Data diperoleh dari idx.co.id diakses pada tanggal 27/08/2022
#Fraksi 1: x<Rp200,00
fraksi1 = ['KIJA.JK', 'LCGP.JK', 'LMPI.JK', 'LPKR.JK']

#Fraksi 2: Rp200,00<= x <Rp500,00
fraksi2 = ['BMTR.JK','BTON.JK','FORU.JK','GEMA.JK']

#Fraksi 3: Rp500,00<= x <Rp2.000,00
fraksi3 = ['AKRA.JK','BRPT.JK','KLBF.JK','MEDC.JK']

#Fraksi 4: Rp2000,00<= x <Rp5000,00
fraksi4 = ['JECC.JK', 'TMAS.JK', 'TPIA.JK', 'UNVR.JK']

#Fraksi 5: >=Rp5000,00
fraksi5 =['INCO.JK','INDF.JK','INTP.JK','UNTR.JK']

In [6]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = fraksi1).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (10916, 8)


In [7]:
df.nunique()

date      2729
open      1014
high      1044
low       1021
close     1441
volume    8684
tic          4
day          5
dtype: int64

In [8]:
print(df['tic'].unique())

['KIJA.JK' 'LCGP.JK' 'LMPI.JK' 'LPKR.JK']


* **sort_values()** berfungsi mengurutkan bingkai data dalam urutan Ascending atau Descending dari kolom yang dilewati.
* **ignore_index** berfungsi untuk menentukan untuk mereset indeks mulai dari nol. Secara default disetel *false*.
* **head()** digunakan untuk mendapatkan n baris pertama.

In [9]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-05,48.857891,49.835049,48.857891,48.236729,96630243,KIJA.JK,0
1,2009-01-05,50.000000,50.000000,50.000000,50.000000,0,LCGP.JK,0
2,2009-01-05,70.000000,70.000000,70.000000,70.000000,0,LMPI.JK,0
3,2009-01-05,596.003723,641.269836,596.003723,581.263245,32545542,LPKR.JK,0
4,2009-01-06,48.857891,52.766521,48.857891,48.236729,226697765,KIJA.JK,1


<a id='3'></a>
# Bagian 4: Memproses Data

Periksa data yang hilang dan melakukan *feature engineering* untuk mengubah data menjadi *state*.
* **Menambahkan indikator teknis**

  Dalam praktik jual-beli, berbagai informasi perlu diperhitungkan, seperti harga historis, kepemilikan saham saat ini, indikator teknis, dll. Indikator teknis yang dapat digunakan antara lain, MACD, RSI, CCI, ADX, Bollinger Bands, dll. 
* **Menambahkan indeks turbulensi**

  Risk-aversion mencerminkan seberapa berani investor melakukan jual-beli saham dengan risiko yang tinggi. Hal ini mempengaruhi strategi trading investor ketika menghadapi tingkat volatilitas pasar yang berbeda. Untuk mengendalikan risiko dalam skenario terburuk, seperti krisis keuangan tahun 1998, FinRL menggunakan indeks turbulensi yang mengukur fluktuasi harga aset yang ekstrem.

In [10]:
INDICATORS = [
    "macd",
    "rsi_30",
    "cci_30"
]

Membersihkan data mentah dari nilai-nilai yang hilang. Sehingga saham dengan data yang hilang dihapus dari daftar.

In [11]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2759, 8)
Successfully added vix
Successfully added turbulence index


In [12]:
processed.nunique()

date           2652
open           1010
high           1038
low            1015
close          1434
volume         8472
tic               4
day               5
macd          10357
rsi_30         7476
cci_30         9824
vix            1441
turbulence     2399
dtype: int64

In [13]:
list_ticker = processed["tic"].unique().tolist()
list_ticker

['KIJA.JK', 'LCGP.JK', 'LMPI.JK', 'LPKR.JK']

In [14]:
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
list_date

['2009-01-05',
 '2009-01-06',
 '2009-01-07',
 '2009-01-08',
 '2009-01-09',
 '2009-01-10',
 '2009-01-11',
 '2009-01-12',
 '2009-01-13',
 '2009-01-14',
 '2009-01-15',
 '2009-01-16',
 '2009-01-17',
 '2009-01-18',
 '2009-01-19',
 '2009-01-20',
 '2009-01-21',
 '2009-01-22',
 '2009-01-23',
 '2009-01-24',
 '2009-01-25',
 '2009-01-26',
 '2009-01-27',
 '2009-01-28',
 '2009-01-29',
 '2009-01-30',
 '2009-01-31',
 '2009-02-01',
 '2009-02-02',
 '2009-02-03',
 '2009-02-04',
 '2009-02-05',
 '2009-02-06',
 '2009-02-07',
 '2009-02-08',
 '2009-02-09',
 '2009-02-10',
 '2009-02-11',
 '2009-02-12',
 '2009-02-13',
 '2009-02-14',
 '2009-02-15',
 '2009-02-16',
 '2009-02-17',
 '2009-02-18',
 '2009-02-19',
 '2009-02-20',
 '2009-02-21',
 '2009-02-22',
 '2009-02-23',
 '2009-02-24',
 '2009-02-25',
 '2009-02-26',
 '2009-02-27',
 '2009-02-28',
 '2009-03-01',
 '2009-03-02',
 '2009-03-03',
 '2009-03-04',
 '2009-03-05',
 '2009-03-06',
 '2009-03-07',
 '2009-03-08',
 '2009-03-09',
 '2009-03-10',
 '2009-03-11',
 '2009-03-

In [15]:
combination = list(itertools.product(list_date,list_ticker))
combination

[('2009-01-05', 'KIJA.JK'),
 ('2009-01-05', 'LCGP.JK'),
 ('2009-01-05', 'LMPI.JK'),
 ('2009-01-05', 'LPKR.JK'),
 ('2009-01-06', 'KIJA.JK'),
 ('2009-01-06', 'LCGP.JK'),
 ('2009-01-06', 'LMPI.JK'),
 ('2009-01-06', 'LPKR.JK'),
 ('2009-01-07', 'KIJA.JK'),
 ('2009-01-07', 'LCGP.JK'),
 ('2009-01-07', 'LMPI.JK'),
 ('2009-01-07', 'LPKR.JK'),
 ('2009-01-08', 'KIJA.JK'),
 ('2009-01-08', 'LCGP.JK'),
 ('2009-01-08', 'LMPI.JK'),
 ('2009-01-08', 'LPKR.JK'),
 ('2009-01-09', 'KIJA.JK'),
 ('2009-01-09', 'LCGP.JK'),
 ('2009-01-09', 'LMPI.JK'),
 ('2009-01-09', 'LPKR.JK'),
 ('2009-01-10', 'KIJA.JK'),
 ('2009-01-10', 'LCGP.JK'),
 ('2009-01-10', 'LMPI.JK'),
 ('2009-01-10', 'LPKR.JK'),
 ('2009-01-11', 'KIJA.JK'),
 ('2009-01-11', 'LCGP.JK'),
 ('2009-01-11', 'LMPI.JK'),
 ('2009-01-11', 'LPKR.JK'),
 ('2009-01-12', 'KIJA.JK'),
 ('2009-01-12', 'LCGP.JK'),
 ('2009-01-12', 'LMPI.JK'),
 ('2009-01-12', 'LPKR.JK'),
 ('2009-01-13', 'KIJA.JK'),
 ('2009-01-13', 'LCGP.JK'),
 ('2009-01-13', 'LMPI.JK'),
 ('2009-01-13', 'LPK

In [16]:
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,48.857891,49.835049,48.857891,48.236729,96630243.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
1,2009-01-05,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
2,2009-01-05,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
3,2009-01-05,LPKR.JK,596.003723,641.269836,596.003723,581.263245,32545542.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
4,2009-01-06,KIJA.JK,48.857891,52.766521,48.857891,48.236729,226697765.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,LPKR.JK,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.290000,0.008272
15996,2019-12-18,KIJA.JK,298.000000,306.000000,292.000000,296.000000,8984100.0,2.0,-5.423601,33.586043,-158.751696,12.580000,0.752572
15997,2019-12-18,LCGP.JK,114.000000,114.000000,114.000000,114.000000,0.0,2.0,-0.000004,44.937077,-158.751696,12.580000,0.752572
15998,2019-12-18,LMPI.JK,101.000000,101.000000,101.000000,101.000000,400.0,2.0,-4.910314,40.464511,-43.939546,12.580000,0.752572


In [17]:
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,48.857891,49.835049,48.857891,48.236729,96630243.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
1,2009-01-05,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
2,2009-01-05,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
3,2009-01-05,LPKR.JK,596.003723,641.269836,596.003723,581.263245,32545542.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
4,2009-01-06,KIJA.JK,48.857891,52.766521,48.857891,48.236729,226697765.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,LPKR.JK,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.290000,0.008272
15996,2019-12-18,KIJA.JK,298.000000,306.000000,292.000000,296.000000,8984100.0,2.0,-5.423601,33.586043,-158.751696,12.580000,0.752572
15997,2019-12-18,LCGP.JK,114.000000,114.000000,114.000000,114.000000,0.0,2.0,-0.000004,44.937077,-158.751696,12.580000,0.752572
15998,2019-12-18,LMPI.JK,101.000000,101.000000,101.000000,101.000000,400.0,2.0,-4.910314,40.464511,-43.939546,12.580000,0.752572


In [18]:
processed_full = processed_full.sort_values(['date','tic'])
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,48.857891,49.835049,48.857891,48.236729,96630243.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
1,2009-01-05,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
2,2009-01-05,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
3,2009-01-05,LPKR.JK,596.003723,641.269836,596.003723,581.263245,32545542.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
4,2009-01-06,KIJA.JK,48.857891,52.766521,48.857891,48.236729,226697765.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,LPKR.JK,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.290000,0.008272
15996,2019-12-18,KIJA.JK,298.000000,306.000000,292.000000,296.000000,8984100.0,2.0,-5.423601,33.586043,-158.751696,12.580000,0.752572
15997,2019-12-18,LCGP.JK,114.000000,114.000000,114.000000,114.000000,0.0,2.0,-0.000004,44.937077,-158.751696,12.580000,0.752572
15998,2019-12-18,LMPI.JK,101.000000,101.000000,101.000000,101.000000,400.0,2.0,-4.910314,40.464511,-43.939546,12.580000,0.752572


In [19]:
processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,48.857891,49.835049,48.857891,48.236729,96630243.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
1,2009-01-05,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
2,2009-01-05,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
3,2009-01-05,LPKR.JK,596.003723,641.269836,596.003723,581.263245,32545542.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
4,2009-01-06,KIJA.JK,48.857891,52.766521,48.857891,48.236729,226697765.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,LPKR.JK,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.290000,0.008272
15996,2019-12-18,KIJA.JK,298.000000,306.000000,292.000000,296.000000,8984100.0,2.0,-5.423601,33.586043,-158.751696,12.580000,0.752572
15997,2019-12-18,LCGP.JK,114.000000,114.000000,114.000000,114.000000,0.0,2.0,-0.000004,44.937077,-158.751696,12.580000,0.752572
15998,2019-12-18,LMPI.JK,101.000000,101.000000,101.000000,101.000000,400.0,2.0,-4.910314,40.464511,-43.939546,12.580000,0.752572


In [20]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,48.857891,49.835049,48.857891,48.236729,96630243.0,0.0,0.000000,100.0,66.666667,39.080002,0.0
1,2009-01-05,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,0.0,0.000000,100.0,66.666667,39.080002,0.0
2,2009-01-05,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,0.0,0.000000,100.0,66.666667,39.080002,0.0
3,2009-01-05,LPKR.JK,596.003723,641.269836,596.003723,581.263245,32545542.0,0.0,0.000000,100.0,66.666667,39.080002,0.0
4,2009-01-06,KIJA.JK,48.857891,52.766521,48.857891,48.236729,226697765.0,1.0,0.000000,100.0,66.666667,38.560001,0.0
5,2009-01-06,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,1.0,0.000000,100.0,66.666667,38.560001,0.0
6,2009-01-06,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,1.0,0.000000,100.0,66.666667,38.560001,0.0
7,2009-01-06,LPKR.JK,626.181152,648.814209,626.181152,602.272766,53837632.0,1.0,0.471367,100.0,66.666667,38.560001,0.0
8,2009-01-07,KIJA.JK,48.857891,51.789364,48.857891,48.236729,203203608.0,2.0,0.000000,100.0,20.000000,43.389999,0.0
9,2009-01-07,LCGP.JK,50.000000,50.000000,50.000000,50.000000,10000.0,2.0,0.000000,100.0,20.000000,43.389999,0.0


In [21]:
processed_full.nunique()

date           2652
tic               4
open           1010
high           1038
low            1015
close          1434
volume         8472
day               5
macd          10357
rsi_30         7476
cci_30         9824
vix            1441
turbulence     2399
dtype: int64

In [22]:
#memperbarui dataframe dengan bentuk harga per lot

processed_full_lot = processed_full[['open','high','low','close']].mul(100)

#drop some columns
processed_full_noOHLC = processed_full.drop(['open','high','low','close'], axis=1)
  
processed_full_lot = pd.concat([processed_full_noOHLC, processed_full_lot], axis=1)

processed_full_lot = processed_full_lot[['date', 'tic', 'open', 'high', 'low', 'close', 'volume', 'day', 'macd', 'rsi_30', 'cci_30', 'vix', 'turbulence']]

processed_full_lot

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,4885.789108,4983.504868,4885.789108,4823.672867,96630243.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
1,2009-01-05,LCGP.JK,5000.000000,5000.000000,5000.000000,5000.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
2,2009-01-05,LMPI.JK,7000.000000,7000.000000,7000.000000,7000.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
3,2009-01-05,LPKR.JK,59600.372314,64126.983643,59600.372314,58126.324463,32545542.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
4,2009-01-06,KIJA.JK,4885.789108,5276.652145,4885.789108,4823.672867,226697765.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,LPKR.JK,24000.000000,24400.000000,23800.000000,24000.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.290000,0.008272
15996,2019-12-18,KIJA.JK,29800.000000,30600.000000,29200.000000,29600.000000,8984100.0,2.0,-5.423601,33.586043,-158.751696,12.580000,0.752572
15997,2019-12-18,LCGP.JK,11400.000000,11400.000000,11400.000000,11400.000000,0.0,2.0,-0.000004,44.937077,-158.751696,12.580000,0.752572
15998,2019-12-18,LMPI.JK,10100.000000,10100.000000,10100.000000,10100.000000,400.0,2.0,-4.910314,40.464511,-43.939546,12.580000,0.752572


<a id='4'></a>
# Bagian 5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style
Proses *training* meliputi mengamati perubahan harga saham, mengambil tindakan dan perhitungan *reward*. Dengan berinteraksi dengan lingkungan pasar, agen pada akhirnya akan memperoleh strategi perdagangan yang dapat memaksimalkan imbalan.

*Environment* pasar dibangun  menggunakan OpenAI Gym yang mensimulasikan pasar saham dengan data historis pasar.

<a id='4.1'></a>
## 5.1. Memisahkan Data
Data dibagi menjadi dua bagian, yaitu bagian pelatihan dan bagian pengujian dengan keterangan sebagai berikut:

Periode data pelatihan: 2009-01-01 hingga 2017-12-31

Periode data perdagangan: 2018-01-01 hingga 2022-08-15

In [23]:
train = data_split(processed_full_lot, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full_lot, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

8628
1980


In [24]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2155,2017-12-28,LPKR.JK,38884.814453,39519.668579,38408.673096,38257.470703,58597272.0,3.0,-28.385327,27.868502,-102.555067,10.18,1.611104
2156,2017-12-29,KIJA.JK,28600.000000,28600.000000,28200.000000,28600.000000,16508800.0,4.0,-3.612968,34.278597,-97.415507,11.04,0.257610
2156,2017-12-29,LCGP.JK,7300.000000,8800.000000,7300.000000,8000.000000,8834300.0,4.0,-1.389227,43.985598,-8.855912,11.04,0.257610
2156,2017-12-29,LMPI.JK,16800.000000,17100.000000,16700.000000,16700.000000,35200.0,4.0,-1.008401,47.293358,-102.476931,11.04,0.257610
2156,2017-12-29,LPKR.JK,38884.814453,39202.243042,38567.388916,38414.910889,39843609.0,4.0,-27.095883,28.442145,-97.849420,11.04,0.257610


In [25]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,KIJA.JK,28600.000000,28600.000000,28000.000000,28400.000000,8749200.0,1.0,-3.498002,32.441935,-113.728129,9.77,1.048363
0,2018-01-02,LCGP.JK,7900.000000,8300.000000,7900.000000,8100.000000,6255600.0,1.0,-1.088875,44.851317,2.335236,9.77,1.048363
0,2018-01-02,LMPI.JK,17100.000000,17200.000000,16700.000000,16800.000000,3400.0,1.0,-1.096968,48.158048,-62.500000,9.77,1.048363
0,2018-01-02,LPKR.JK,38884.814453,39043.527222,38408.673096,38100.030518,26017693.0,1.0,-24.700852,27.966124,-88.363095,9.77,1.048363
1,2018-01-03,KIJA.JK,28600.000000,28600.000000,28000.000000,28400.000000,15693500.0,2.0,-3.471229,32.441935,-111.111111,9.15,0.089529


In [26]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 4, State Space: 21


In [27]:
buy_cost_list = [0.0023] * stock_dimension
sell_cost_list = [0.0054] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 100000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

<a id='4.2'></a>
## 5.2. Lingkungan untuk Pelatihan *(Training)*



In [28]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Bagian 6: Melatih Agen DRL
* Algoritma DRL dibangun dengan menggunakan **Stable Baselines 3**. 

* FinRL mencakup algoritma DRL standar yang disempurnakan, seperti DQN, DDPG, DDPG Multi-Agen, PPO, SAC, A2C, dan TD3. FinRL juga mengizinkan pengguna untuk merancang algoritma DRL sendiri dengan mengadaptasi algoritma DRL yang telah disebutkan.

In [29]:
agent = DRLAgent(env = env_train)

**Agent Training: 5 algorithms (A2C, PPO, TD3, SAC, DDPG)**

<a id='5.1'></a>
##6.1. Agen 1: A2C


In [30]:
A2C_PARAMS = {
    "n_steps": 150, 
    "ent_coef": 0.0001, 
    "learning_rate": 0.0014,
}
model_a2c = agent.get_model("a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 150, 'ent_coef': 0.0001, 'learning_rate': 0.0014}
Using cuda device


In [31]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

--------------------------------------
| time/                 |            |
|    fps                | 346        |
|    iterations         | 100        |
|    time_elapsed       | 43         |
|    total_timesteps    | 15000      |
| train/                |            |
|    entropy_loss       | -5.61      |
|    explained_variance | -1.24e-05  |
|    learning_rate      | 0.0014     |
|    n_updates          | 99         |
|    policy_loss        | 1.31e+03   |
|    reward             | -1022.5247 |
|    std                | 0.982      |
|    value_loss         | 7.65e+06   |
--------------------------------------
day: 2156, episode: 10
begin_total_asset: 100000000.00
end_total_asset: 169941289.69
total_reward: 69941289.69
total_cost: 49085492.47
total_trades: 6592
Sharpe: 0.388
-------------------------------------
| time/                 |           |
|    fps                | 358       |
|    iterations         | 200       |
|    time_elapsed       | 83        |
|    total_timeste

<a id='5.2'></a>
##6.2. Agen 2: PPO

In [32]:
PPO_PARAMS = {
    "n_steps": 50000,
    "ent_coef": 0.7,
    "learning_rate": 0.0016,
    "batch_size": 1750,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 50000, 'ent_coef': 0.7, 'learning_rate': 0.0016, 'batch_size': 1750}
Using cuda device


/usr/local/lib/python3.7/dist-packages/stable_baselines3/ppo/ppo.py:147: UserWarning: You have specified a mini-batch size of 1750, but because the `RolloutBuffer` is of size `n_steps * n_envs = 50000`, after every 28 untruncated mini-batches, there will be a truncated mini-batch of size 1000
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=50000 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"


In [33]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

day: 2156, episode: 30
begin_total_asset: 100000000.00
end_total_asset: 140129980.83
total_reward: 40129980.83
total_cost: 62492893.75
total_trades: 8347
Sharpe: 0.283
day: 2156, episode: 40
begin_total_asset: 100000000.00
end_total_asset: 106276869.49
total_reward: 6276869.49
total_cost: 63334213.21
total_trades: 8359
Sharpe: 0.146
----------------------------------
| time/              |           |
|    fps             | 373       |
|    iterations      | 1         |
|    time_elapsed    | 133       |
|    total_timesteps | 50000     |
| train/             |           |
|    reward          | 1.5141324 |
----------------------------------


<a id='5.3'></a>
##6.3. Agen 3: TD3

In [34]:
TD3_PARAMS = {
    "batch_size": 7000, 
    "buffer_size": 10000, 
    "learning_rate": 0.9  
}
model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 7000, 'buffer_size': 10000, 'learning_rate': 0.9}
Using cuda device


In [35]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000)

day: 2156, episode: 50
begin_total_asset: 100000000.00
end_total_asset: 96081019.49
total_reward: -3918980.51
total_cost: 1582559.86
total_trades: 6558
Sharpe: 0.154
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 85        |
|    time_elapsed    | 101       |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | 25.5      |
|    critic_loss     | 1.98e+05  |
|    learning_rate   | 0.9       |
|    n_updates       | 6471      |
|    reward          | 36.764362 |
----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 74        |
|    time_elapsed    | 230       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | 12.4      |
|    critic_loss     | 2.06e+05  |
|    learning_rate   | 0.9       |
|    n_updates       | 15099 

<a id='5.4'></a>
##6.4. Agen 4: SAC

In [36]:
SAC_PARAMS = {
    "ent_coef": 1.25,
    "learning_rate": 0.0001,
    "batch_size": 4000,
    "buffer_size": 5000,
    "learning_starts": 14,
    
}
model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'ent_coef': 1.25, 'learning_rate': 0.0001, 'batch_size': 4000, 'buffer_size': 5000, 'learning_starts': 14}
Using cuda device


In [37]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=30000)

---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 61       |
|    time_elapsed    | 139      |
|    total_timesteps | 8628     |
| train/             |          |
|    actor_loss      | 2.62e+04 |
|    critic_loss     | 3.14e+05 |
|    ent_coef        | 1.25     |
|    learning_rate   | 0.0001   |
|    n_updates       | 8613     |
|    reward          | 9.052605 |
---------------------------------
day: 2156, episode: 80
begin_total_asset: 100000000.00
end_total_asset: 83822007.24
total_reward: -16177992.76
total_cost: 229467.15
total_trades: 4312
Sharpe: 0.127
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 61       |
|    time_elapsed    | 278      |
|    total_timesteps | 17256    |
| train/             |          |
|    actor_loss      | 2.16e+04 |
|    critic_loss     | 1.53e+05 |
|    ent_coef        | 1.25     |
|    learning_rate

<a id='5.5'></a>
##6.5. Agen 5: DDPG

In [38]:
DDPG_PARAMS = {
    "batch_size": 1250, 
    "buffer_size": 50000, 
    "learning_rate": 0.0003
}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 1250, 'buffer_size': 50000, 'learning_rate': 0.0003}
Using cuda device


In [39]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=60000)

day: 2156, episode: 90
begin_total_asset: 100000000.00
end_total_asset: 103067543.39
total_reward: 3067543.39
total_cost: 229468.39
total_trades: 4312
Sharpe: 0.175
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 123       |
|    time_elapsed    | 69        |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | -3.35e+04 |
|    critic_loss     | 2.62e+08  |
|    learning_rate   | 0.0003    |
|    n_updates       | 6471      |
|    reward          | 24.576412 |
----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 109       |
|    time_elapsed    | 157       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | -3.38e+03 |
|    critic_loss     | 1.71e+07  |
|    learning_rate   | 0.0003    |
|    n_updates       | 15099  

<a id='6'></a>
# Bagian 7: Melakukan Jual-Beli

<a id='6.1'></a>
## 7.1. Performa di Dalam Sampel

Asumsikan modal awal adalah Rp100.000.000.

In [40]:
data_risk_indicator = processed_full_lot[(processed_full_lot.date<TRAIN_END_DATE) & (processed_full_lot.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [41]:
insample_risk_indicator.vix.describe()

count    2157.000000
mean       18.666583
std         7.813415
min         9.140000
25%        13.370000
50%        16.299999
75%        21.540001
max        56.650002
Name: vix, dtype: float64

In [42]:
insample_risk_indicator.vix.quantile(0.996)

48.84047967529307

In [43]:
insample_risk_indicator.turbulence.describe()

count    2157.000000
mean        4.121638
std        12.797609
min         0.000000
25%         0.391273
50%         1.345604
75%         3.532035
max       364.269034
Name: turbulence, dtype: float64

In [44]:
insample_risk_indicator.turbulence.quantile(0.996)

79.19871216654998

<a id='6.2'></a>
##7.2. Performa di Luar Sampel

FinRL dapat diatur dengan cara melakukan pelatihan ulang secara berkala, misalnya, pelatihan ulang setiap tiga bulan, bulanan, atau mingguan. 

Pada notebook ini, *hyperparameter* hanya diatur satu kali dengan menggunakan data sampel selama periode yang telah ditetapkan. Sehingga, terjadi beberapa peluruhan alfa seiring dengan perpanjangan tanggal perdagangan.

Banyak *hyperparameter* – mis. learning rate, mempengaruhi proses pembelajaran dan biasanya ditentukan dengan menguji beberapa variasi.

In [45]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70, risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

<a id='6.3'></a>
##7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi

<a id='6.3.1'></a>
###7.3.1. Agen 1: A2C

In [46]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment = e_trade_gym)

hit end!


<a id='6.3.2'></a>
###7.3.2. Agen 2: PPO

In [47]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    environment = e_trade_gym)

hit end!


<a id='6.3.3'></a>
###7.3.3. Agen 3: TD3

In [48]:
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3, 
    environment = e_trade_gym)

hit end!


<a id='6.3.4'></a>
###7.3.4. Agen 4: SAC

In [49]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

hit end!


<a id='6.3.5'></a>
###7.3.5. Agen 5: DDPG

In [50]:
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym)

hit end!


<a id='7'></a>
# Bagian 8: Performa Backtesting
Backtesting memiliki peran kunci dalam mengevaluasi kinerja strategi perdagangan. Alat backtesting otomatis lebih disukai karena dapat meminimalisir kesalahan manusia. Backtesting dapat dilakukan dengan menggunakan paket Quantopian pyfolio untuk menguji strategi perdagangan pada notebook ini. Backtesting tersebut mudah digunakan dan terdiri dari berbagai plot yang memberikan gambaran komprehensif tentang kinerja strategi perdagangan.

<a id='7.1'></a>
## 8.1 Status BackTesting


###8.1.1 Agen A2C

In [51]:
print("===========Get Backtest Results of A2C Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_a2c)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_a2c_"+now+'.csv')

===========Get Backtest Results of A2C Model===========
Annual return         -0.255764
Cumulative returns    -0.440238
Annual volatility      0.307571
Sharpe ratio          -0.807163
Calmar ratio          -0.521490
Stability              0.747000
Max drawdown          -0.490448
Omega ratio            0.847851
Sortino ratio         -1.075491
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.857565
Daily value at risk   -0.039735
dtype: float64


###8.1.2 Agen ppo

In [52]:
print("===========Get Backtest Results of PPO Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ppo)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ppo_"+now+'.csv')

===========Get Backtest Results of PPO Model===========
Annual return         -0.061116
Cumulative returns    -0.116510
Annual volatility      0.158224
Sharpe ratio          -0.320640
Calmar ratio          -0.341544
Stability              0.220480
Max drawdown          -0.178941
Omega ratio            0.935396
Sortino ratio         -0.476114
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.116749
Daily value at risk   -0.020136
dtype: float64


###8.1.3 Agen TD3

In [53]:
print("===========Get Backtest Results of TD3 Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_td3)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_td3_"+now+'.csv')

===========Get Backtest Results of TD3 Model===========
Annual return         -0.078421
Cumulative returns    -0.148211
Annual volatility      0.194023
Sharpe ratio          -0.324973
Calmar ratio          -0.238707
Stability              0.020528
Max drawdown          -0.328524
Omega ratio            0.941479
Sortino ratio         -0.461543
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.900983
Daily value at risk   -0.024695
dtype: float64


###8.1.4 Agen SAC

In [54]:
print("===========Get Backtest Results of SAC Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_sac)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_sac_"+now+'.csv')

===========Get Backtest Results of SAC Model===========
Annual return         -0.231193
Cumulative returns    -0.403359
Annual volatility      0.276416
Sharpe ratio          -0.814945
Calmar ratio          -0.488186
Stability              0.607914
Max drawdown          -0.473575
Omega ratio            0.861352
Sortino ratio         -1.141200
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.010837
Daily value at risk   -0.035719
dtype: float64


###8.1.5 Agen DDPG

In [55]:
print("===========Get Backtest Results of DDPG Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ddpg)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ddpg_"+now+'.csv')

===========Get Backtest Results of DDPG Model===========
Annual return         -0.118205
Cumulative returns    -0.218936
Annual volatility      0.217375
Sharpe ratio          -0.471686
Calmar ratio          -0.277278
Stability              0.073277
Max drawdown          -0.426304
Omega ratio            0.917498
Sortino ratio         -0.683925
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.042196
Daily value at risk   -0.027794
dtype: float64


###8.1.6. Baseline Stats

In [56]:
#baseline stats
print("===========Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^JKII", 
        start = df_account_value_a2c.loc[0,'date'],
        end = df_account_value_a2c.loc[len(df_account_value_a2c)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')

===========Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (478, 8)
Annual return         -0.041972
Cumulative returns    -0.078113
Annual volatility      0.185511
Sharpe ratio          -0.138819
Calmar ratio          -0.175216
Stability              0.093175
Max drawdown          -0.239543
Omega ratio            0.977818
Sortino ratio         -0.189296
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.900451
Daily value at risk   -0.023474
dtype: float64
